In [1]:
import sys
import scipy.io
import h5py
import numpy as np
import os, sys, time, subprocess, h5py, argparse, logging, pickle
import numpy as np
import pandas as pd
from os.path import join as oj
from copy import deepcopy
from scipy.ndimage import imread
from scipy.misc import imresize
from PIL import Image
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path                   
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import data.data as data
import re

%matplotlib inline
%load_ext autoreload
%autoreload 2

# preprocess raw data

In [3]:

def read_conditions(fname):
    c_raw = pd.read_csv(fname, delimiter=',')
    c = pd.DataFrame() #pd.read_csv(oj('data', 'star_polymer_conditions.csv'), delimiter=',')
    c['fnames'] = c_raw.fnames.astype(str)
    c['conc'] = c_raw.conc_solution # mol/L
    c['solvent_glyc'] = [int(x[:2]) for x in c_raw.solvent_glyc]# percent glycerol, 5% ethanol, 85% water
    c['pressure'] = c_raw.pressure # mba  r
    c['time'] = c_raw.time # time delivered for (s)
    c['zmin'] = zz = [float(re.sub("[^0-9*.-]", "", zz).split('*')[0]) for zz in c_raw.zz] # physical width = physical height# max zscale
    c['zmax'] = zz = [float(re.sub("[^0-9*.-]", "", zz).split('*')[1]) for zz in c_raw.zz] # physical width = physical height# max zscale
    c['width_physical'] = [float(re.sub("[^0-9*.]", "", xy).split('*')[0]) for xy in c_raw.xy] # physical width = physical height
    c['R'] = c_raw.R # based on scan lines
    c['C'] = c_raw.C # based on scan lines
    return c
conditions = read_conditions(oj('data', 'star_polymer_conditions.csv'))

In [4]:
from lib.igg import binarywave
def read_ims(data_raw_dir, conditions):


data_raw_dir = 'data/star_polymer/star_polymer_raw_ibw'
read_ims(data_raw_dir, conditions)

In [ ]:
# load dset (takes a few seconds)
dset = Star_dset()
# dset.conditions.head()
dset.conditions

# times = initial dev. time (how long we deliver soln)
# z-scale = processing of image (so that we can capture variation at better res)
# x_y_size  = res of image
# scan lines (left to right)
# scan_points_per_line: res of each line
# important inputs: sol_concentration	pressure	time